In [2]:
import os

import pandas as pd

In [21]:
import datetime

import fasttext
import keras_core as keras
import numpy as np
import pandas as pd
from keras_core import Sequential
from keras_core.src.layers import Dense
from keras_core.models import load_model
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf

from tensorboard.plugins.hparams import api as hp

## import fastTest model

In [13]:
import os
#embedded_path = './../../'

#contents = os.listdir(embedded_path)
#print(contents)
ft = fasttext.load_model('../../data/embedding_data/cc.fr.300.bin')

['notebooks', 'tmp', 'tests', '.env', 'ml', 'pyproject.toml', 'README.md', 'Init.ipynb', 'accounts.db', 'src', 'data', '.env.template', 'docs', 'model_weights', '__init__.py', 'poetry.lock', 'log']


## Configurations

In [77]:
params_grid = {
    "model" : "nlp",
    "embedder" : "fasttext",
    "embedding data" : "cc.fr.300.bin",
    "dataset" : "all_data.csv",
    "batch_size" : 300
}

In [98]:
data_folder_src = './../../data/processed/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'
date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f'deepwoke_{params_grid["embedder"]}_{params_grid["model"]}_{params_grid["dataset"]}_{date_str}'

log_dir = f"../../log/fit/{experiment_name}"
model_weight_dst = f'../../model_weights/{experiment_name}-model.keras'

# loading dataset

In [84]:

ds = pd.read_csv(data_path_src).iloc[:, 1:].dropna()
ds.rename(columns={"text": "full_text"}, inplace=True)
ds.rename(columns={"hateful": "label"}, inplace=True)
ds


,full_text,label
0,rt @user personnage ainsi q tte la clique gauc...,1
1,@user @user @user bah oui t'as raison l'autre ...,1
2,moi j'ai personne à qui m'adresser pour réclam...,1
3,@user @user c...est vrai que les anchois sont ...,0
4,eh les renois faut se réveiller la @url,0
...,...,...
16893,L'islam a Ã©levÃ© la Femme en islam une femme ...,0
16894,L'islam considÃ¨re que la femme est un joyau. ...,0
16895,L'islam valorise la femme sur beaucoup d'aspec...,0
16896,"Le Coran a libÃ©rÃ© la femme, l'a honorÃ© en t...",0


## text to vector

In [85]:
def text_to_vector(text):
    words = text.split(' ')
    word_vectors = [ft.get_word_vector(word) for word in words if word in ft.words]
    if not word_vectors:
        return np.zeros(300)
    return np.mean(word_vectors, axis=0)

In [86]:
tqdm.pandas(desc='Loading data')
ds['vector'] = ds['full_text'].progress_apply(text_to_vector)

X = np.vstack(ds['vector'])
y = ds['label'].values

Loading data: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16898/16898 [17:43<00:00, 15.89it/s]


## building and training the model

### callbacks

In [99]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [100]:

class EarlyStoppingLogging(tf.keras.callbacks.Callback):
    def __init__(self, early_stopping_callback, log_dir):
        super().__init__()
        self.early_stopping = early_stopping_callback
        self.stopped_epoch = 0
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        if self.early_stopping.stopped_epoch > 0:
            self.stopped_epoch = self.early_stopping.stopped_epoch
            with self.writer.as_default():
                tf.summary.scalar('early_stopping_epoch', self.stopped_epoch, step=epoch)
                self.writer.flush()


In [101]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_logging_callback = EarlyStoppingLogging(early_stopping, log_dir)

In [102]:
callback = [
    early_stopping,
    tensorboard_callback,
    hp.KerasCallback(log_dir, params_grid),
    early_stopping_logging_callback,
]

### training the model

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, input_dim=300, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(34, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train,
          y_train,
          epochs=500,
          batch_size = params_grid['batch_size'],
          callbacks=callback,
          validation_data=(X_test, y_test)
          )

Epoch 1/500
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.6386 - loss: 0.6449 - val_accuracy: 0.7406 - val_loss: 0.5139
Epoch 2/500
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7560 - loss: 0.5106 - val_accuracy: 0.7591 - val_loss: 0.4863
Epoch 3/500
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7764 - loss: 0.4750 - val_accuracy: 0.7909 - val_loss: 0.4591
Epoch 4/500
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7867 - loss: 0.4601 - val_accuracy: 0.7915 - val_loss: 0.4499
Epoch 5/500
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7937 - loss: 0.4453 - val_accuracy: 0.7956 - val_loss: 0.4395
Epoch 6/500
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8010 - loss: 0.4344 - val_accuracy: 0.7885 - val_loss: 0.4371
Epoch 7/500
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8019 - loss: 0.4260 - val_accuracy: 0.7965 - val_loss: 0.4310
Epoch 8/500
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8125 - loss: 0.4104 - val_accuracy: 0.8072 - 

# save model

In [104]:
model.save(model_weight_dst)

### testing with custom sentence

In [105]:
#model_weight_dst = f'../../model_weights/fasttext_nlp_20240613-131608-model.keras'
new_model = load_model(model_weight_dst)

In [107]:
text_nice = [
    "la bonjour je suis un type sympas", 
    "bonjour ça va les gens",
    "j'aime pas trop ce mec",
    "cette meuf est cool",
    "les meuf sont trop cool",
    "les mecs sont des gens stylé",
    "le mec est stylé",
    "nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse",
    "les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste",
    "Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions"
    
]

text_no_nice = [
    "regarde moi cette connasse",
    "les femme leur place c'est à la cuisine",
    "femme au volant, accident",
    "les femmes on peux pas leur faire confiance",
    "Pour une transphobe son passing est pas ouf",
    "Elle l’ouvre quand même beaucoup celle là pour quelqu’un qu’est pas capable d’éduquer ses enfants",
    "Dis donc, c’est pas parce que t’es mal baisée que tu peux tout te permettre ????",
    "Moi ce que j’aime bien chez les filles comme toi c’est que t’es pas du genre prise de tête",
    "les femmes c'est comme le fromage, quand elles puent c'est qu'elles sont bonnes",
    "Oh les grosses tepus",
    "Faut pas t'étonner si tu te fais violer vu comment t'es habillée",
    "Tu deverais avoir des enfants maintenant avant que ce soit trop tard",
    "Vous seriez une collaboratrice idéal pour notre entreprise, malheureusement nous cherchons quelqu’un disponible sur le long terme, et nous craignons qu’à votre âge des changements familiaux ne vous permettent plus de vous investir autant que besoins"
]
wordsvec_nice = [np.expand_dims(text_to_vector(text), axis=0) for text in text_nice]
wordsvec_no_nice = [np.expand_dims(text_to_vector(text), axis=0) for text in text_no_nice]

print("nice output")
for i in range(0, len(wordsvec_nice)):
    prediction = new_model.predict(wordsvec_nice[i], verbose=False)
    print(f"{text_nice[i]} : {prediction > 0.5}")
    print(f"{prediction}")


print("\n\nnot nice output")
for i in range(0, len(wordsvec_no_nice)):
    prediction =  new_model.predict(wordsvec_no_nice[i], verbose=False)
    print(f"{text_no_nice[i]} : {prediction > 0.5}")
    print(f"{prediction}")

nice output
la bonjour je suis un type sympas : [[False]]
[[0.12051824]]
bonjour ça va les gens : [[ True]]
[[0.9994973]]
j'aime pas trop ce mec : [[False]]
[[5.6627454e-09]]
cette meuf est cool : [[ True]]
[[0.9886635]]
les meuf sont trop cool : [[ True]]
[[0.9993337]]
les mecs sont des gens stylé : [[ True]]
[[0.9997663]]
le mec est stylé : [[ True]]
[[0.9988927]]
nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse : [[False]]
[[0.06172562]]
les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste : [[ True]]
[[0.82011175]]
Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions : [[False]]
[[3.9226315e-06]]


not nice output
regarde moi cette connasse : [[ True]]
[[0.9982065]]
les femme leur place c'est à la cuisine : [[ True]]
[[0.64545226]]
femme au volant, accident : [[ True]]
[[0.99988973]]
les femmes on peux pas leur faire confiance : [[False]]
[[0